In [1]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, CONNECTIONS_CONFIG, SMOL_LM_135M

cfg = CONNECTIONS_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 1000

/Users/brianfitzgerald/Documents/GitHub/smolmodels/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-12 05:07:12,646	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

2025-04-12 05:07:16.112 | INFO     | trl_wrapper.trainer_wrapper:__init__:285 - Setting padding side to: right
2025-04-12 05:07:16.125 | INFO     | trl_wrapper.trainer_wrapper:init_model:296 - Using device: mps
2025-04-12 05:07:16.126 | INFO     | trl_wrapper.trainer_wrapper:init_model:310 - Loading model HuggingFaceTB/SmolLM2-135M-Instruct with attn_impl: sdpa


In [3]:
from model.reasoning import ConnectionsDataModule

In [4]:
conn = ConnectionsDataModule(wrapper.tokenizer, wrapper.config)

2025-04-12 05:07:22.046 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/connections_data_module


In [5]:
wrapper.init_data_module()

2025-04-12 05:07:39.939 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:336 - Using chat template override: smollmv2
2025-04-12 05:07:39.939 | INFO     | trl_wrapper.wrapper_config:__init__:145 - Cache dir: ../dataset_caches/connections_data_module
Map: 100%|██████████| 100000/100000 [00:05<00:00, 18153.70 examples/s]


In [6]:
wrapper.init_trainer()

2025-04-12 05:12:01.799 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:373 - Saving output to: ./runs/04-12-5-12-998514-smollm2-135m-instruct-
2025-04-12 05:12:01.800 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:396 - Initializing trainer, run_name: 04-12-5-12-998514-smollm2-135m-instruct-, wandb project: qwen-connections-grpo
2025-04-12 05:12:01.800 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:479 - Using vllm: False


In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [11]:
wrapper.train()

2025-04-12 05:20:31.876 | INFO     | trl_wrapper.trainer_wrapper:train:644 - Starting training.
2025-04-12 05:20:54.939 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.0]
2025-04-12 05:20:54.940 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-04-12 05:20:54.940 | INFO     | model.reasoning:soft_group_reward:189 - Soft accuracy scores: [0.0, 0.0]
2025-04-12 05:20:54.941 | INFO     | model.reasoning:group_size_reward:226 - Group size rewards: [0.0, 0.0]
2025-04-12 05:20:54.941 | INFO     | model.reasoning:hard_group_reward:198 - Generations:
2025-04-12 05:20:54.941 | INFO     | model.reasoning:hard_group_reward:200 - Prompt:--------------------
2025-04-12 05:20:54.941 | INFO     | model.reasoning:hard_group_reward:201 - hair, makeup, props, wardrobe, mercury, sky, sparks, liberty, cent, chainz, pac, savage, digit, dog, mitt, piggy
2025-04-12 05:20:54.942 | INFO     | model.reasoning:hard_group_reward:20

Step,Training Loss


2025-04-12 05:23:59.938 | INFO     | model.reasoning:xmlcount_reward_func:59 - XML count rewards: [0.0, 0.0]
2025-04-12 05:23:59.938 | INFO     | model.reasoning:strict_format_reward_func:105 - Strict format rewards: [0.0, 0.0]
2025-04-12 05:23:59.939 | INFO     | model.reasoning:soft_group_reward:189 - Soft accuracy scores: [0.0, 0.0]
2025-04-12 05:23:59.939 | INFO     | model.reasoning:group_size_reward:226 - Group size rewards: [0.0, 0.0]
2025-04-12 05:23:59.940 | INFO     | model.reasoning:hard_group_reward:198 - Generations:
2025-04-12 05:23:59.940 | INFO     | model.reasoning:hard_group_reward:200 - Prompt:--------------------
2025-04-12 05:23:59.940 | INFO     | model.reasoning:hard_group_reward:201 - family, flush, jelly, we, curses, darn, rats, shoot, atlas, hermes, nike, paris, cardamom, clove, coriander, cumin
2025-04-12 05:23:59.940 | INFO     | model.reasoning:hard_group_reward:202 - Generation:--------------------
2025-04-12 05:23:59.941 | INFO     | model.reasoning:hard_

KeyboardInterrupt: 